# Chapter 13 Copy Control

## EXERCISES SECTION 13.1.1

### Exercise 13.1:
What is a copy constructor? When is it used?

A:

Copy constructor is a special constructor that is used to create a new object as a copy of an existing object. It is used when we want to copy an object, e.g. when an object is passed by value to a function or returned from a function.

### Exercise 13.2: 
Explain why the following declaration is illegal:
```
Sales_data::Sales_data(Sales_data rhs);
```

A:

The parameter must be a reference, as a non-reference parameter would try to copy the object before the copy constructor is defined.


### Exercise 13.3: 
What happens when we copy a `StrBlob`? What about `StrBlobPtrs`?

A:

Syntesized copy constructor is used. Pointers kept in these objects are copied (and in case of `shared_ptr`, `use_count` is increased), but objects that these pointers point to are not copied. Copies point to the same objects as copied ones.

### Exercise 13.4: 
Assuming `Point` is a class type with a public copy constructor, 
identify each use of the copy constructor in this program fragment:
```
Point global;
Point foo_bar(Point arg)
{
    Point local = arg, *heap = new Point(global);
    *heap = local;
    Point pa[ 4 ] = { local, *heap };
    return *heap;
}
```

A:

- copying from argument in `Point arg`
- copying `arg` in  `Point local = arg`
- copying `global` in  `*heap = new Point(global)`
- two times when copying `local` and `heap` in `Point pa[ 4 ] = { local, *heap }`
- copying `*heap` when returning `return *heap`

### Exercise 13.5: 
Given the following sketch of a class, write a copy constructor that
copies all the members. Your constructor should dynamically allocate a new `string`
(§ 12.1.2, p. 458) and copy the object to which `ps` points, rather than copying `ps` itself.
```
class HasPtr {
    public:
        HasPtr(const std::string &s = std::string()):
        ps(new std::string(s)), i(0) { }
    private:
        std::string *ps;
        int
        i;
};
```

In [1]:
class HasPtr {
    public:
        HasPtr(const std::string &s = std::string()): ps(new std::string(s)), i(0) { }
        HasPtr(const HasPtr &orig): ps(new std::string(*orig.ps)), i(orig.i) { }
        std::string *ps;
        int i;
};

In [2]:
HasPtr ptr("text");
HasPtr ptr2(ptr);
*ptr.ps = "text_old";

In [3]:
*ptr.ps

"text_old"

In [4]:
*ptr2.ps

"text"

## EXERCISES SECTION 13.1.2

### Exercise 13.6: 
What is a copy-assignment operator?
When is this operator used?
What does the synthesized copy-assignment operator do? When is it synthesized?

A:

Copy-assignment operator is a `=` operator used when object is already created and we assign an object of the same type to it. 

Synthesized copy-assignment assigns each member of the right-hand object to corresponding member of the left-hand object. It return a reference to the left-hand object. This operator typically performs a shallow copy, where only the values of the data members are copied, but not the dynamically allocated memory or resources.

Operator is synthesized if it's the class does not define it's own.

### Exercise 13.7: 
What happens when we assign one `StrBlob` to another? What about `StrBlobPtrs`?

A:

Same as in 13.3

### Exercise 13.8: 
Write the assignment operator for the `HasPtr` class from exercise 13.5
in § 13.1.1 (p. 499). As with the copy constructor, your assignment operator should
copy the object to which `ps` points.

In [1]:
class HasPtr {
    public:
        HasPtr(const std::string &s = std::string()): ps(new std::string(s)), i(0) { }
        HasPtr(const HasPtr &orig): ps(new std::string(*orig.ps)), i(orig.i) { }
        HasPtr& operator=(const HasPtr& rhs) {
            if (this != &rhs) {
                auto new_ps = new std::string(*rhs.ps);
                delete ps;
                ps = new_ps;
                i = rhs.i;
            }
            return *this;
        }
        std::string *ps;
        int i;
};

In [2]:
HasPtr ptr("text");
HasPtr ptr2("text2");
ptr2 = ptr;
*ptr.ps = "text_old";

In [3]:
*ptr.ps

"text_old"

In [4]:
*ptr2.ps

"text"

## EXERCISES SECTION 13.1.3

### Exercise 13.9: 
What is a destructor? What does the synthesized destructor do? When
is a destructor synthesized?

A:

Destructor frees the resources used by an object. It's syntesized for any class that doesn't define it's constructor.

### Exercise 13.10: 
What happens when a `StrBlob` object is destroyed? What about a
`StrBlobPtr`?

A:

When a `StrBlob` object is destroyed, the `shared_ptr`'s use_count is decreased, which in turn destroyes the pointer and the underlying data as use_count reaches 0.

When a `StrBlobPtr` object is destroyed, it does not delete any underlying data or shared resources.
It simply destroys the object itself.

### Exercise 13.11: 
Add a destructor to your `HasPtr` class from the previous exercises.

In [1]:
class HasPtr {
    public:
        HasPtr(const std::string &s = std::string()): ps(new std::string(s)), i(0) { }
        HasPtr(const HasPtr &orig): ps(new std::string(*orig.ps)), i(orig.i) { }
        HasPtr& operator=(const HasPtr& rhs) {
            if (this != &rhs) {
                auto new_ps = new std::string(*rhs.ps);
                delete ps;
                ps = new_ps;
                i = rhs.i;
            }
            return *this;
        }
        ~HasPtr() { delete ps; }
        std::string *ps;
        int i;
};

### Exercise 13.12: 
How many destructor calls occur in the following code fragment?
```
bool fcn(const Sales_data *trans, Sales_data accum)
{
    Sales_data item1(*trans), item2(accum);
    return item1.isbn() != item2.isbn();
}
```

A:

Two destructor calls: one for `item1` and one for `item2`.

### Exercise 13.13:
A good way to understand copy-control members and constructors is
to define a simple class with these members in which each member prints its name:
```
struct X {
    X() {std::cout << "X()" << std::endl;}
    X(const X&) {std::cout << "X(const X&)" << std::endl;}
};
```
Add the copy-assignment operator and destructor to `X` and write a program using `X`
objects in various ways: Pass them as nonreference and reference parameters; 
dynamically allocate them; put them in containers; and so forth. Study the output until you
are certain you understand when and why each copy-control member is used. As you
read the output, remember that the compiler can omit calls to the copy constructor.

In [1]:
#include <iostream>

struct X {
    X() {std::cout << "X()" << std::endl;}
    X(const X&) {std::cout << "X(const X&)" << std::endl;}
    X& operator=(const X&) {std::cout << "X& operator=(const X&)" << std::endl; return *this;}
    ~X() {std::cout << "~X()" << std::endl;}
};

In [2]:
X x1, x2(x1);

X()
X(const X&)


In [3]:
x1 = x2;

X& operator=(const X&)


In [4]:
std::vector<X> v;

In [5]:
v.push_back(x1);

X(const X&)


In [6]:
v.push_back(x2);

X(const X&)
X(const X&)
~X()


In [7]:
v.push_back(x1);

X(const X&)
X(const X&)
X(const X&)
~X()
~X()


In [8]:
v.push_back(x2);

X(const X&)


In [9]:
v.size()

4

In [10]:
v.capacity()

4

In [11]:
v.push_back(x2);

X(const X&)
X(const X&)
X(const X&)
X(const X&)
X(const X&)
~X()
~X()
~X()
~X()


In [12]:
v.capacity()

8

In [12]:
v.pop_back()

~X()


In [12]:
X* px = new X;

X()


In [15]:
x1 = *px;

X& operator=(const X&)


In [16]:
delete px;

~X()


In [17]:
v.emplace_back(x1);

X(const X&)


## EXERCISES SECTION 13.1.4

### Exercise 13.14: 
Assume that `numbered` is a class with a default constructor that 
generates a unique serial number for each object, which is stored in a data member named
`mysn`. Assuming `numbered` uses the synthesized copy-control members and given
the following function:
```
void f (numbered s) { cout << s.mysn << endl; }
```
what output does the following code produce?
```
numbered a, b = a, c = b;
f(a); f(b); f(c);
```

A:

The same serial number three times

### Exercise 13.15: 
Assume `numbered` has a copy constructor that generates a new serial
number. Does that change the output of the calls in the previous exercise? If so, why?
What output gets generated?

A:

Serial numbers will be different this time, as custom copy constructor won't copy the serial number, just generate new one.

### Exercise 13.16: 
What if the parameter in `f` were `const numbered&`?
Does that change the output? If so, why? What output gets generated?

A:

using reference instead of copying argument to `f` will skip additional copying, so output will be different if we use custom copy constructor, or the same if we use default copy constructor

### Exercise 13.17: 
Write versions of `numbered` and `f` corresponding to the previous
three exercises and check whether you correctly predicted the output.

In [1]:
#include <iostream>
using std::cout, std::endl;

In [2]:
struct numbered {
    numbered() { 
        static int serial_number = 0;
        mysn = ++serial_number;
    }
    int mysn;
};

In [3]:
void f (numbered s) { cout << s.mysn << endl; }

In [4]:
numbered a, b = a, c = b;
f(a); f(b); f(c);

1
1
1


In [1]:
#include <iostream>
using std::cout, std::endl;

In [2]:
struct numbered {
    numbered() = default;
    numbered(const numbered &n) {
        static int serial_number = 0;
        mysn = ++serial_number;
    }
    int mysn;
};

In [3]:
void f (numbered s) { cout << s.mysn << endl; }

In [4]:
numbered a, b = a, c = b;
f(a); f(b); f(c);

3
4
5


In [1]:
#include <iostream>
using std::cout, std::endl;

In [2]:
struct numbered {
    numbered() { 
        static int serial_number = 0;
        mysn = ++serial_number;
    }
    int mysn;
};

In [3]:
void f (const numbered& s) { cout << s.mysn << endl; }

In [4]:
numbered a, b = a, c = b;
f(a); f(b); f(c);

1
1
1


## EXERCISES SECTION 13.1.6

### Exercise 13.18:
Define an `Employee` class that contains an employee name and a
unique employee identifier. Give the class a default constructor and a constructor that
takes a `string` representing the employee’s name. Each constructor should generate
a unique ID by incrementing a static data member.

In [1]:
#include <string>
using std::string;

In [2]:
struct Employee {
    Employee(): id(++curr_id) { }
    Employee(const string &n): name(n), id(++curr_id) { }

    string name;
    inline static int curr_id = 0;
    int id;
}

In [3]:
Employee e1, e2("mark");

In [4]:
e1.name

""

In [5]:
e2.name

"mark"

In [6]:
e1.id

1

In [7]:
e2.id

2

### Exercise 13.19: 
Does your `Employee` class need to define its own versions of the
copy-control members? If so, why? If not, why not? Implement whatever copy-control
members you think `Employee` needs.

A:

If we want the `name` to be copied, but `id` to be unique, we need to define copy-control members ourselves

In [1]:
#include <string>
using std::string;

In [2]:
struct Employee {
    Employee(): id(++curr_id) { }
    Employee(const string &n): name(n), id(++curr_id) { }
    
    Employee(const Employee &e): name(e.name), id(++curr_id) { }
    Employee& operator=(const Employee &e)
    {
        if (this != &e)
        {
            name = e.name;
            id = ++curr_id;
        }
        
        return *this;
    }

    string name;
    inline static int curr_id = 0;
    int id;
}

### Exercise 13.20: 
Explain what happens when we copy, assign, or destroy objects of our
`TextQuery` and `QueryResult` classes from § 12.3 (p. 484).

A:

Smart pointers are used in both of these classes, so they when copied or assigned, their use_count will be increased. When destroyed, use_count will be decreased. Other members (`string word` and `map querymap`) will copied or destroyed.

### Exercise 13.21: 
Do you think the `TextQuery` and `QueryResult` classes need to 
define their own versions of the copy-control members? If so, why? If not, why not?
Implement whichever copy-control operations you think these classes require.

A:

They don't need to define their own copy-control members, as there's no usage of their members (we never change them, only read them) that would indicate we need to copy the memory under their pointers. Copying the pointers is enough.

## EXERCISES SECTION 13.2

### Exercise 13.22: 
Assume that we want `HasPtr` to behave like a value. That is, each
object should have its own copy of the `string` to which the objects point. We’ll show
the definitions of the copy-control members in the next section. However, you already
know everything you need to know to implement these members. Write the `HasPtr`
copy constructor and copy-assignment operator before reading on.

A:

Done in 13.8

## EXERCISES SECTION 13.2.1

### Exercise 13.23: 
Compare the copy-control members that you wrote for the solutions
to the previous section’s exercises to the code presented here. Be sure you understand
the differences, if any, between your code and ours.

A:

Done in 13.8

### Exercise 13.24: 
What would happen if the version of `HasPtr` in this section didn’t
define a destructor? What if `HasPtr` didn’t define the copy constructor?

A:

Without a destructor, memory under `ps` would not be freed.  
Without a copy constructor, only pointer `ps` would be copied, not the underlying memory.


### Exercise 13.25: 
Assume we want to define a version of `StrBlob` that acts like a value.
Also assume that we want to continue to use a `shared_ptr` so that our `StrBlobPtr`
class can still use a `weak_ptr` to the vector. Your revised class will need a copy
constructor and copy-assignment operator but will not need a destructor. Explain what
the copy constructor and copy-assignment operators must do. Explain why the class
does not need a destructor.

A:

Copy constructor and copy-assignment operator must copy the data under the `shared_ptr` `data`. Destructor doesn't need to be defined, as the default destructor will delete the `shared_ptr`, decreasing it's `use_count` and removing the memory if it's the last pointer.

### Exercise 13.26: 
Write your own version of the `StrBlob` class described in the previous exercise.

In [1]:
using std::initializer_list, std::string, std::vector, std::shared_ptr, std::make_shared;

In [2]:
class StrBlob {
    public:
        typedef std::vector<std::string>::size_type size_type;
        typedef std::shared_ptr<std::vector<std::string>> ptr;
        StrBlob(): data(make_shared<vector<string>>()) { }
        StrBlob(initializer_list<string> il): data(make_shared<vector<string>>(il)) { }
        StrBlob(const StrBlob &orig): data(make_shared<vector<string>>(*orig.data)) { }
        StrBlob& operator=(const StrBlob& rhs) {
            if (this != &rhs) {
                data = make_shared<vector<string>>(*rhs.data);
            }
            return *this;
        }
        size_type size() const { return data->size(); }
        bool empty() const { return data->empty(); }
        void push_back(const std::string &t) {data->push_back(t);}
        ptr get_data() { return data; }
    private:
        ptr data;
};

In [6]:
StrBlob blob({"elo", "1"});

In [9]:
*blob.get_data()

{ "elo", "1" }

In [10]:
StrBlob bl(blob);

In [11]:
*bl.get_data()

{ "elo", "1" }

In [13]:
bl.push_back("test")

In [14]:
*bl.get_data()

{ "elo", "1", "test" }

In [15]:
*blob.get_data()

{ "elo", "1" }

In [16]:
bl = blob;

@0x7f70cd476048

In [19]:
*bl.get_data()

{ "elo", "1" }

In [20]:
*blob.get_data()

{ "elo", "1" }

In [21]:
bl.push_back("test")

In [22]:
*bl.get_data()

{ "elo", "1", "test" }

In [23]:
*blob.get_data()

{ "elo", "1" }

## EXERCISES SECTION 13.2.2

### Exercise 13.27: 
Define your own reference-counted version of `HasPtr`.

In [1]:
class HasPtr {
    public:
        HasPtr(const std::string &s = std::string()): ps(new std::string(s)), i(0), use(new std::size_t(1)) { }
        HasPtr(const HasPtr &orig): ps(orig.ps), i(orig.i), use(orig.use) { ++*use; }
        HasPtr& operator=(const HasPtr& rhs) {
            if (this != &rhs) {
                ++*rhs.use;
                if (--*use == 0) {
                    delete ps;
                    delete use;
                }
                ps = rhs.ps;
                i = rhs.i;
                use = rhs.use;
            }
            return *this;
        }
        ~HasPtr() { 
            if (--*use == 0) {
                delete ps;
                delete use;
            }
        }
        std::string *ps;
        int i;
        std::size_t *use;
};

In [2]:
HasPtr ptr("test");

In [3]:
*ptr.ps

"test"

In [4]:
*ptr.use

1

In [5]:
HasPtr pt2(ptr);

In [6]:
*pt2.ps

"test"

In [7]:
*pt2.use

2

In [8]:
*ptr.use

2

In [9]:
ptr = pt2

@0x7fe095bad028

In [10]:
*pt2.use

2

In [11]:
*ptr.use

2

### Exercise 13.28: 
Given the following classes, implement a default constructor and the
necessary copy-control members.
```
(a) class TreeNode {
    private:
        std::string value;
        int count;
        TreeNode *left;
        TreeNode *right;
};
```
```
(b) class BinStrTree {
    private:
        TreeNode *root;
};
```

In [1]:
class TreeNode {
    public:
        TreeNode() = default;
        TreeNode(const TreeNode &orig);
        ~TreeNode() { delete left; delete right; }
        TreeNode& operator=(const TreeNode& rhs) {
            value = rhs.value;
            count = rhs.count;
            if (rhs.left == nullptr)
                left = nullptr;
            else
            {
                auto left_temp = new TreeNode(*rhs.left);
                delete left;
                left = left_temp;
            }
            if (rhs.right == nullptr)
                right = nullptr;
            else
            {
                auto right_temp = new TreeNode(*rhs.right);
                delete right;
                right = right_temp;
            }
            return *this;
        }
    private:
        std::string value;
        int count;
        TreeNode *left;
        TreeNode *right;
};

In [2]:
TreeNode::TreeNode(const TreeNode &orig) : value(orig.value), count(orig.count) {
    if (orig.left != nullptr)
        left = new TreeNode(*orig.left);
    else
        left = nullptr;
    if (orig.right != nullptr)
        right = new TreeNode(*orig.right);
    else
        right = nullptr;
}

In [3]:
class BinStrTree {
    public:
        BinStrTree(TreeNode &r): root(&r) { };
        BinStrTree(const BinStrTree &orig) : root(new TreeNode(*orig.root)) { };
        ~BinStrTree() { delete root; }
        BinStrTree& operator=(const BinStrTree& rhs) {
            root = rhs.root;
            return *this;
        }
    private:
        TreeNode *root;
};

## EXERCISES SECTION 13.3

### Exercise 13.29: 
Explain why the calls to `swap` inside `swap(HasPtr&, HasPtr&)` do not cause a recursion loop.

### Exercise 13.30: 
Write and test a `swap` function for your valuelike version of `HasPtr`.
Give your `swap` a `print` statement that notes when it is executed.

### Exercise 13.31: 
Give your class a `<` operator and define a `vector` of `HasPtrs`. Give
that `vector` some elements and then sort the `vector`. Note when `swap` is called.

### Exercise 13.32: 
Would the pointerlike version of `HasPtr` benefit from defining a `swap`
function? If so, what is the benefit? If not, why not?

## EXERCISES SECTION 13.4

### Exercise 13.33: 
Why is the parameter to the save and remove members of `Message`
a `Folder&`? Why didn’t we define that parameter as `Folder`? Or `const Folder&`?

### Exercise 13.34: 
Write the `Message` class as described in this section.

### Exercise 13.35: 
What would happen if `Message` used the synthesized versions of the
copy-control members?

### Exercise 13.36: 
Design and implement the corresponding `Folder` class. That class
should hold a `set` that points to the `Messages` in that `Folder`.

### Exercise 13.37: 
Add members to the `Message` class to insert or remove a given
`Folder*` into folders. These members are analogous to `Folder`’s `addMsg` and
`remMsg` operations.

### Exercise 13.38: 
We did not use `copy` and `swap` to define the `Message` assignment
operator. Why do you suppose this is so?

## EXERCISES SECTION 13.5

### Exercise 13.39: 
Write your own version of `StrVec`, including versions of `reserve`,
`capacity` (§ 9.4, p. 356), and `resize` (§ 9.3.5, p. 352).

### Exercise 13.40: 
Add a constructor that takes an `initializer_list<string>` to
your `StrVec` class.

### Exercise 13.41: 
Why did we use postfix increment in the call to construct inside
`push_back`? What would happen if it used the prefix increment?

### Exercise 13.42: 
Test your `StrVec` class by using it in place of the `vector<string>`
in your `TextQuery` and `QueryResult` classes (§ 12.3, p. 484).

### Exercise 13.43: 
Rewrite the free member to use `for_each` and a lambda (§ 10.3.2,
p. 388) in place of the `for` loop to destroy the elements. Which implementation do
you prefer, and why?

### Exercise 13.44: 
Write a class named `String` that is a simplified version of the library
`string` class. Your class should have at least a default constructor and a constructor
that takes a pointer to a C-style string. Use an `allocator` to allocate memory that
your `String` class uses.

## EXERCISES SECTION 13.6.1

### Exercise 13.45: 
Distinguish between an rvalue reference and an lvalue reference.

### Exercise 13.46: 
Which kind of reference can be bound to the following initializers?
```
int f();
vector<int> vi(100);
int? r1 = f();
int? r2 = vi[0];
int? r3 = r1;
int? r4 = vi[0] * f();
```

### Exercise 13.47: 
Give the copy constructor and copy-assignment operator in your
`String` class from exercise 13.44 in § 13.5 (p. 531) a statement that prints a `message`
each time the function is executed.

### Exercise 13.48: 
Define a `vector<String>` and call `push_back` several times on that
`vector`. Run your program and see how often `String`s are copied.

## EXERCISES SECTION 13.6.2

### Exercise 13.49: 
Add a move constructor and move-assignment operator to your
`StrVec`, `String`, and `Message` classes.

### Exercise 13.50: 
Put print statements in the move operations in your `String` class and
rerun the program from exercise 13.48 in § 13.6.1 (p. 534) that used a `vector<String>`
to see when the copies are avoided.

### Exercise 13.51: 
Although `unique_ptrs` cannot be copied, in § 12.1.5 (p. 471) we
wrote a clone function that returned a `unique_ptr` by value. Explain why that 
function is legal and how it works.

### Exercise 13.52: 
Explain in detail what happens in the assignments of the `HasPtr` 
objects on page 541. In particular, describe step by step what happens to values of `hp`,
`hp2`, and of the `rhs` parameter in the `HasPtr` assignment operator.

### Exercise 13.53: 
As a matter of low-level efficiency, the `HasPtr` assignment operator
is not ideal. Explain why. Implement a copy-assignment and move-assignment 
operator for `HasPtr` and compare the operations executed in your new move-assignment
operator versus the copy-and-swap version.

### Exercise 13.54: 
What would happen if we defined a `HasPtr` move-assignment operator
but did not change the copy-and-swap operator? Write code to test your answer.

## EXERCISES SECTION 13.6.3

### Exercise 13.55:
Add an rvalue reference version of `push_back` to your `StrBlob`.

### Exercise 13.56: 
What would happen if we defined `sorted` as:
```
Foo Foo::sorted() const & {
    Foo ret(*this);
    return ret.sorted();
}
```

### Exercise 13.57: 
What if we defined `sorted` as:
```
Foo Foo::sorted() const & { return Foo(*this).sorted(); }
```

### Exercise 13.58: 
Write versions of class `Foo` with print statements in their `sorted`
functions to test your answers to the previous two exercises.